## The purpose of this notebook is to test the `etm` object. 

### 1. Load train data

In [2]:
from sklearn.datasets import fetch_20newsgroups

train_data = fetch_20newsgroups(subset='train') # from sklearn
test_data = fetch_20newsgroups(subset='test')

In [3]:
import sys, os
sys.path.append('..')

from src.preprocessor import Preprocessor

prep = Preprocessor()
bow_train = prep.fit_transform(train_data.data)

print( prep.get_feature_names()[:5] )
print( prep.transform(['aaron aaron aaron ab']).toarray() )

print( prep.get_feature_names()[-5:] )
print( prep.transform(['zuma zuma zuma']).toarray() )

['aaron', 'ab', 'ability', 'abortion', 'absolute']
[[3 1 0 ... 0 0 0]]
['zealand', 'zip', 'zone', 'zoo', 'zuma']
[[0 0 0 ... 0 0 3]]


### 2. Load the `etm` object

In [4]:
import argparse
import torch
import pickle 
import numpy as np 
import os 
import math 
import random 
import sys
import matplotlib.pyplot as plt 
import scipy.io

from torch import nn, optim
from torch.nn import functional as F

sys.path.append('..')
import data
from src.etm import ETM
from utils import nearest_neighbors, get_topic_coherence, get_topic_diversity

np.random.seed(2019)
torch.manual_seed(2019)
if torch.cuda.is_available():
    torch.cuda.manual_seed(2019)
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### init model and optimization params

In [5]:
# model param
num_topics = 50
vocab_size = len( prep.get_feature_names() )
enc_drop=0
model = ETM(num_topics, vocab_size, enc_drop=enc_drop).to(device)

# optimizer params
lr = 0.005 # learning rate
wdecay=1.2e-6
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wdecay)
batch_size = 1000
log_interval = 2
ckpt = 'model_ckpt'

### Training logic

In [7]:
def sparse2torch(sparse, device):
    return torch.from_numpy( sparse.toarray() ).float().to(device)

def train_one_epoch(bow_train, epoch, verbose=1):
    model.train()

    acc_loss = 0
    acc_kl_theta_loss = 0
    cnt = 0

    indices = torch.randperm(bow_train.shape[0])
    indices = torch.split(indices, 1000)

    for idx, ind in enumerate(indices):
        optimizer.zero_grad()
        model.zero_grad()
        
        data_batch = sparse2torch(bow_train[ind], device)
        sums = data_batch.sum(1).unsqueeze(1)
        normalized_data_batch = data_batch / sums
        
        recon_loss, kld_theta = model(data_batch, normalized_data_batch)
        total_loss = recon_loss + kld_theta
        total_loss.backward()

        optimizer.step()

        acc_loss += torch.sum(recon_loss).item()
        acc_kl_theta_loss += torch.sum(kld_theta).item()
        cnt += 1

        if idx % log_interval == 0 and idx > 0:
            cur_loss = round(acc_loss / cnt, 2) 
            cur_kl_theta = round(acc_kl_theta_loss / cnt, 2) 
            cur_real_loss = round(cur_loss + cur_kl_theta, 2)
            
            if verbose:
                print('Epoch: {} .. batch: {}/{} .. LR: {} .. KL_theta: {} .. Rec_loss: {} .. NELBO: {}'.format(
                    epoch, idx, len(indices), optimizer.param_groups[0]['lr'], cur_kl_theta, cur_loss, cur_real_loss))

    cur_loss = round(acc_loss / cnt, 2) 
    cur_kl_theta = round(acc_kl_theta_loss / cnt, 2) 
    cur_real_loss = round(cur_loss + cur_kl_theta, 2)
    
    if verbose:
        print('*'*100)
        print('Epoch----->{} .. LR: {} .. KL_theta: {} .. Rec_loss: {} .. NELBO: {}'.format(
                epoch, optimizer.param_groups[0]['lr'], cur_kl_theta, cur_loss, cur_real_loss))
        print('*'*100)

In [9]:
epochs = 1000
for i in range(epochs):
    train_one_epoch(bow_train, i)

Epoch: 0 .. batch: 2/12 .. LR: 0.005 .. KL_theta: 0.03 .. Rec_loss: 666.58 .. NELBO: 666.61
Epoch: 0 .. batch: 4/12 .. LR: 0.005 .. KL_theta: 0.02 .. Rec_loss: 673.43 .. NELBO: 673.45
Epoch: 0 .. batch: 6/12 .. LR: 0.005 .. KL_theta: 0.01 .. Rec_loss: 692.44 .. NELBO: 692.45
Epoch: 0 .. batch: 8/12 .. LR: 0.005 .. KL_theta: 0.01 .. Rec_loss: 692.8 .. NELBO: 692.81
Epoch: 0 .. batch: 10/12 .. LR: 0.005 .. KL_theta: 0.01 .. Rec_loss: 677.51 .. NELBO: 677.52
****************************************************************************************************
Epoch----->0 .. LR: 0.005 .. KL_theta: 0.01 .. Rec_loss: 668.68 .. NELBO: 668.69
****************************************************************************************************
Epoch: 1 .. batch: 2/12 .. LR: 0.005 .. KL_theta: 0.0 .. Rec_loss: 641.19 .. NELBO: 641.19
Epoch: 1 .. batch: 4/12 .. LR: 0.005 .. KL_theta: 0.0 .. Rec_loss: 642.2 .. NELBO: 642.2
Epoch: 1 .. batch: 6/12 .. LR: 0.005 .. KL_theta: 0.0 .. Rec_loss: 653.38 .. 

Epoch: 11 .. batch: 8/12 .. LR: 0.005 .. KL_theta: 0.5 .. Rec_loss: 635.1 .. NELBO: 635.6
Epoch: 11 .. batch: 10/12 .. LR: 0.005 .. KL_theta: 0.52 .. Rec_loss: 637.57 .. NELBO: 638.09
****************************************************************************************************
Epoch----->11 .. LR: 0.005 .. KL_theta: 0.53 .. Rec_loss: 646.65 .. NELBO: 647.18
****************************************************************************************************
Epoch: 12 .. batch: 2/12 .. LR: 0.005 .. KL_theta: 0.69 .. Rec_loss: 645.99 .. NELBO: 646.68
Epoch: 12 .. batch: 4/12 .. LR: 0.005 .. KL_theta: 0.69 .. Rec_loss: 634.17 .. NELBO: 634.86
Epoch: 12 .. batch: 6/12 .. LR: 0.005 .. KL_theta: 0.79 .. Rec_loss: 635.66 .. NELBO: 636.45
Epoch: 12 .. batch: 8/12 .. LR: 0.005 .. KL_theta: 0.82 .. Rec_loss: 636.1 .. NELBO: 636.92
Epoch: 12 .. batch: 10/12 .. LR: 0.005 .. KL_theta: 0.83 .. Rec_loss: 642.31 .. NELBO: 643.14
*******************************************************************

Epoch: 22 .. batch: 10/12 .. LR: 0.005 .. KL_theta: 4.46 .. Rec_loss: 621.04 .. NELBO: 625.5
****************************************************************************************************
Epoch----->22 .. LR: 0.005 .. KL_theta: 4.48 .. Rec_loss: 622.28 .. NELBO: 626.76
****************************************************************************************************
Epoch: 23 .. batch: 2/12 .. LR: 0.005 .. KL_theta: 4.78 .. Rec_loss: 611.26 .. NELBO: 616.04
Epoch: 23 .. batch: 4/12 .. LR: 0.005 .. KL_theta: 4.78 .. Rec_loss: 619.51 .. NELBO: 624.29
Epoch: 23 .. batch: 6/12 .. LR: 0.005 .. KL_theta: 4.73 .. Rec_loss: 622.67 .. NELBO: 627.4
Epoch: 23 .. batch: 8/12 .. LR: 0.005 .. KL_theta: 4.75 .. Rec_loss: 621.1 .. NELBO: 625.85
Epoch: 23 .. batch: 10/12 .. LR: 0.005 .. KL_theta: 4.8 .. Rec_loss: 618.08 .. NELBO: 622.88
****************************************************************************************************
Epoch----->23 .. LR: 0.005 .. KL_theta: 4.82 .. Rec_loss: 6

Epoch: 34 .. batch: 2/12 .. LR: 0.005 .. KL_theta: 7.23 .. Rec_loss: 598.96 .. NELBO: 606.19
Epoch: 34 .. batch: 4/12 .. LR: 0.005 .. KL_theta: 7.12 .. Rec_loss: 596.77 .. NELBO: 603.89
Epoch: 34 .. batch: 6/12 .. LR: 0.005 .. KL_theta: 7.15 .. Rec_loss: 598.31 .. NELBO: 605.46
Epoch: 34 .. batch: 8/12 .. LR: 0.005 .. KL_theta: 7.11 .. Rec_loss: 600.84 .. NELBO: 607.95
Epoch: 34 .. batch: 10/12 .. LR: 0.005 .. KL_theta: 7.09 .. Rec_loss: 603.32 .. NELBO: 610.41
****************************************************************************************************
Epoch----->34 .. LR: 0.005 .. KL_theta: 7.17 .. Rec_loss: 609.42 .. NELBO: 616.59
****************************************************************************************************
Epoch: 35 .. batch: 2/12 .. LR: 0.005 .. KL_theta: 7.26 .. Rec_loss: 593.15 .. NELBO: 600.41
Epoch: 35 .. batch: 4/12 .. LR: 0.005 .. KL_theta: 7.27 .. Rec_loss: 602.36 .. NELBO: 609.63
Epoch: 35 .. batch: 6/12 .. LR: 0.005 .. KL_theta: 7.23 .. Rec_l

Epoch: 45 .. batch: 6/12 .. LR: 0.005 .. KL_theta: 8.19 .. Rec_loss: 603.62 .. NELBO: 611.81
Epoch: 45 .. batch: 8/12 .. LR: 0.005 .. KL_theta: 8.1 .. Rec_loss: 595.73 .. NELBO: 603.83
Epoch: 45 .. batch: 10/12 .. LR: 0.005 .. KL_theta: 8.1 .. Rec_loss: 601.33 .. NELBO: 609.43
****************************************************************************************************
Epoch----->45 .. LR: 0.005 .. KL_theta: 8.06 .. Rec_loss: 596.25 .. NELBO: 604.31
****************************************************************************************************
Epoch: 46 .. batch: 2/12 .. LR: 0.005 .. KL_theta: 8.25 .. Rec_loss: 608.82 .. NELBO: 617.07
Epoch: 46 .. batch: 4/12 .. LR: 0.005 .. KL_theta: 8.38 .. Rec_loss: 611.42 .. NELBO: 619.8
Epoch: 46 .. batch: 6/12 .. LR: 0.005 .. KL_theta: 8.42 .. Rec_loss: 608.11 .. NELBO: 616.53
Epoch: 46 .. batch: 8/12 .. LR: 0.005 .. KL_theta: 8.34 .. Rec_loss: 605.95 .. NELBO: 614.29
Epoch: 46 .. batch: 10/12 .. LR: 0.005 .. KL_theta: 8.27 .. Rec_los

...

Epoch: 947 .. batch: 6/12 .. LR: 0.005 .. KL_theta: 13.88 .. Rec_loss: 589.49 .. NELBO: 603.37
Epoch: 947 .. batch: 8/12 .. LR: 0.005 .. KL_theta: 13.79 .. Rec_loss: 576.32 .. NELBO: 590.11
Epoch: 947 .. batch: 10/12 .. LR: 0.005 .. KL_theta: 13.76 .. Rec_loss: 574.55 .. NELBO: 588.31
****************************************************************************************************
Epoch----->947 .. LR: 0.005 .. KL_theta: 13.69 .. Rec_loss: 573.96 .. NELBO: 587.65
****************************************************************************************************
Epoch: 948 .. batch: 2/12 .. LR: 0.005 .. KL_theta: 13.69 .. Rec_loss: 574.25 .. NELBO: 587.94
Epoch: 948 .. batch: 4/12 .. LR: 0.005 .. KL_theta: 13.86 .. Rec_loss: 594.71 .. NELBO: 608.57
Epoch: 948 .. batch: 6/12 .. LR: 0.005 .. KL_theta: 13.77 .. Rec_loss: 586.12 .. NELBO: 599.89
Epoch: 948 .. batch: 8/12 .. LR: 0.005 .. KL_theta: 13.66 .. Rec_loss: 574.11 .. NELBO: 587.77
Epoch: 948 .. batch: 10/12 .. LR: 0.005 .. KL_th

Epoch: 958 .. batch: 10/12 .. LR: 0.005 .. KL_theta: 13.72 .. Rec_loss: 574.07 .. NELBO: 587.79
****************************************************************************************************
Epoch----->958 .. LR: 0.005 .. KL_theta: 13.73 .. Rec_loss: 574.98 .. NELBO: 588.71
****************************************************************************************************
Epoch: 959 .. batch: 2/12 .. LR: 0.005 .. KL_theta: 13.75 .. Rec_loss: 589.1 .. NELBO: 602.85
Epoch: 959 .. batch: 4/12 .. LR: 0.005 .. KL_theta: 13.87 .. Rec_loss: 579.01 .. NELBO: 592.88
Epoch: 959 .. batch: 6/12 .. LR: 0.005 .. KL_theta: 13.91 .. Rec_loss: 578.46 .. NELBO: 592.37
Epoch: 959 .. batch: 8/12 .. LR: 0.005 .. KL_theta: 13.85 .. Rec_loss: 578.33 .. NELBO: 592.18
Epoch: 959 .. batch: 10/12 .. LR: 0.005 .. KL_theta: 13.77 .. Rec_loss: 574.66 .. NELBO: 588.43
****************************************************************************************************
Epoch----->959 .. LR: 0.005 .. KL_theta: 1

Epoch: 970 .. batch: 2/12 .. LR: 0.005 .. KL_theta: 13.99 .. Rec_loss: 600.28 .. NELBO: 614.27
Epoch: 970 .. batch: 4/12 .. LR: 0.005 .. KL_theta: 13.9 .. Rec_loss: 586.93 .. NELBO: 600.83
Epoch: 970 .. batch: 6/12 .. LR: 0.005 .. KL_theta: 13.78 .. Rec_loss: 576.88 .. NELBO: 590.66
Epoch: 970 .. batch: 8/12 .. LR: 0.005 .. KL_theta: 13.75 .. Rec_loss: 574.81 .. NELBO: 588.56
Epoch: 970 .. batch: 10/12 .. LR: 0.005 .. KL_theta: 13.72 .. Rec_loss: 574.81 .. NELBO: 588.53
****************************************************************************************************
Epoch----->970 .. LR: 0.005 .. KL_theta: 13.71 .. Rec_loss: 573.34 .. NELBO: 587.05
****************************************************************************************************
Epoch: 971 .. batch: 2/12 .. LR: 0.005 .. KL_theta: 13.82 .. Rec_loss: 570.47 .. NELBO: 584.29
Epoch: 971 .. batch: 4/12 .. LR: 0.005 .. KL_theta: 13.87 .. Rec_loss: 572.14 .. NELBO: 586.01
Epoch: 971 .. batch: 6/12 .. LR: 0.005 .. KL_thet

Epoch: 981 .. batch: 6/12 .. LR: 0.005 .. KL_theta: 13.69 .. Rec_loss: 572.13 .. NELBO: 585.82
Epoch: 981 .. batch: 8/12 .. LR: 0.005 .. KL_theta: 13.74 .. Rec_loss: 578.19 .. NELBO: 591.93
Epoch: 981 .. batch: 10/12 .. LR: 0.005 .. KL_theta: 13.7 .. Rec_loss: 575.81 .. NELBO: 589.51
****************************************************************************************************
Epoch----->981 .. LR: 0.005 .. KL_theta: 13.69 .. Rec_loss: 571.19 .. NELBO: 584.88
****************************************************************************************************
Epoch: 982 .. batch: 2/12 .. LR: 0.005 .. KL_theta: 13.61 .. Rec_loss: 583.79 .. NELBO: 597.4
Epoch: 982 .. batch: 4/12 .. LR: 0.005 .. KL_theta: 13.71 .. Rec_loss: 582.29 .. NELBO: 596.0
Epoch: 982 .. batch: 6/12 .. LR: 0.005 .. KL_theta: 13.79 .. Rec_loss: 581.63 .. NELBO: 595.42
Epoch: 982 .. batch: 8/12 .. LR: 0.005 .. KL_theta: 13.8 .. Rec_loss: 576.42 .. NELBO: 590.22
Epoch: 982 .. batch: 10/12 .. LR: 0.005 .. KL_theta:

Epoch: 992 .. batch: 10/12 .. LR: 0.005 .. KL_theta: 13.72 .. Rec_loss: 574.29 .. NELBO: 588.01
****************************************************************************************************
Epoch----->992 .. LR: 0.005 .. KL_theta: 13.76 .. Rec_loss: 574.07 .. NELBO: 587.83
****************************************************************************************************
Epoch: 993 .. batch: 2/12 .. LR: 0.005 .. KL_theta: 13.42 .. Rec_loss: 576.79 .. NELBO: 590.21
Epoch: 993 .. batch: 4/12 .. LR: 0.005 .. KL_theta: 13.49 .. Rec_loss: 564.46 .. NELBO: 577.95
Epoch: 993 .. batch: 6/12 .. LR: 0.005 .. KL_theta: 13.64 .. Rec_loss: 567.03 .. NELBO: 580.67
Epoch: 993 .. batch: 8/12 .. LR: 0.005 .. KL_theta: 13.74 .. Rec_loss: 567.08 .. NELBO: 580.82
Epoch: 993 .. batch: 10/12 .. LR: 0.005 .. KL_theta: 13.79 .. Rec_loss: 575.22 .. NELBO: 589.01
****************************************************************************************************
Epoch----->993 .. LR: 0.005 .. KL_theta: 

### Evaluation
Nearest neighbours

In [ ]:
# def visualize_nearest_neighbours(self, queries):
#     """
#     sample queries:
#     ---------------
#         queries = ['andrew', 'computer', 'sports', 'religion', 'man', 'love', 
#                     'intelligence', 'money', 'politics', 'health', 'people', 'family']
#     """
#     for word in queries:
#         print('word: {} .. neighbors: {}'.format(
#             word, nearest_neighbors(word, self.model.word_embed.weight, self.prep.get_feature_names())))
#     print('#'*100)

In [15]:
queries = ['andrew', 'computer', 'sports', 'religion', 'man', 'love', 
            'intelligence', 'money', 'politics', 'health', 'people', 'family']
for word in queries:
    print('word: {} .. neighbors: {}'.format(
        word, nearest_neighbors(word, model.word_embed.weight, prep.get_feature_names())))
print('#'*100)

vectors:  (3318, 300)
query:  (300,)
word: andrew .. neighbors: ['andrew', 'informatik', 'cam', 'uni', 'burns', 'demon', 'ac', 'useragent', 'van', 'nuntius', 'mellon', 'xxdate', 'carnegie', 'umd', 'nz', 'cmu', 'tu', 'simon', 'andy', 'eng']
vectors:  (3318, 300)
query:  (300,)
word: computer .. neighbors: ['computer', 'engineering', 'computing', 'fax', 'phone', 'mail', 'department', 'test', 'standard', 'comp', 'center', 'gmt', 'kit', 'message', 'internet', 'systems', 'reply', 'design', 'bbs', 'info']
vectors:  (3318, 300)
query:  (300,)
word: sports .. neighbors: ['sports', 'stats', 'sport', 'ticket', 'montreal', 'shots', 'scoring', 'coach', 'gm', 'uwaterloo', 'player', 'club', 'scored', 'rush', 'playoffs', 'traded', 'season', 'maynard', 'cubs', 'minnesota']
vectors:  (3318, 300)
query:  (300,)
word: religion .. neighbors: ['religion', 'religious', 'religions', 'belief', 'beliefs', 'god', 'moral', 'faith', 'conclusion', 'christianity', 'christian', 'church', 'islam', 'evil', 'beings', '

In [48]:
with torch.no_grad():
    topics_words=[]
    gammas = model.get_word_topic_matrix()
    for k in range(50):
        gamma = gammas[k]
        top_words = list(gamma.cpu().numpy().argsort()[-10+1:][::-1])
        topic_words = [prep.get_feature_names()[a] for a in top_words]
        topics_words.append(' '.join(topic_words))
        print('Topic {}: {}'.format(k, topic_words))

Topic 0: ['good', 'time', 'thing', 'lot', 'work', 'find', 'make', 'long', 'pretty']
Topic 1: ['car', 'bike', 'dod', 'cars', 'pitt', 'gordon', 'engine', 'banks', 'article']
Topic 2: ['mr', 'president', 'people', 'government', 'clinton', 'jobs', 'work', 'time', 'program']
Topic 3: ['year', 'game', 'baseball', 'team', 'games', 'players', 'season', 'good', 'runs']
Topic 4: ['window', 'file', 'program', 'server', 'motif', 'mit', 'widget', 'entry', 'output']
Topic 5: ['writes', 'article', 'nntp', 'posting', 'host', 'david', 'university', 'news', 'distribution']
Topic 6: ['opinions', 'michael', 'mike', 'corp', 'paul', 'writes', 'scott', 'disclaimer', 'corporation']
Topic 7: ['people', 'fbi', 'fire', 'stratus', 'children', 'started', 'batf', 'koresh', 'happened']
Topic 8: ['good', 'time', 'writes', 'make', 'thing', 'article', 'find', 'people', 'long']
Topic 9: ['msg', 'food', 'medical', 'health', 'disease', 'patients', 'research', 'people', 'medicine']
Topic 10: ['good', 'time', 'people', 'mak

Evaluating the model, it seems like it hasnt decided where to put these obserations

In [96]:
model.eval();
row = sparse2torch(bow_train, device)[1000]
row = row/row.sum()
model.encode( row )[0].max()

tensor(0.0552, device='cuda:0', grad_fn=<MaxBackward1>)

In [26]:
with open(ckpt, 'wb') as f:
    torch.save(model, f)

/home/vinitrinh/anaconda3/envs/gr2/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ETM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/vinitrinh/anaconda3/envs/gr2/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Dropout. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/vinitrinh/anaconda3/envs/gr2/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/vinitrinh/anaconda3/envs/gr2/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon l

In [46]:
prep

Preprocessor(max_df=0.7, min_df=0.005)

In [47]:
from joblib import dump, load
dump(prep, 'prep.joblib') 

['prep.joblib']

In [49]:
with open(ckpt, 'rb') as f:
    model = torch.load(f)
model = model.to(device)

In [30]:
model.word_embed

Linear(in_features=300, out_features=3318, bias=False)